# Predicting March Madness:
- ***Last Updated***: 04/01/2025
- ***Author*** : Jack Thorp

#### Dependencies:

In [8]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from itertools import combinations

#### Building Dataset for Analysis Function:
- [Dataset Information from Kaggle March Madness Competition](https://www.kaggle.com/competitions/march-machine-learning-mania-2025/data)
- Used DFs for Analysis:
    - Teams ID file (*"MTeams.csv"*)
    - Regular Season Matchup Statistics (*"MRegularSeasonDetailedResults.csv"*)
    - Compilation of Basketball Rankings (*"MMasseyOrdinals.csv"*)
    - NCAA Tournament Seeds (*"MNCAATourneySeeds.csv"*)

In [2]:
# === Function to Build Dataset ===
def build_team_season_df(
    data_dir: str,
    start_year: int = 2015,
    end_year: int = 2024,
    ranking_day: int = 133,
    systems: list = ["POM"]
) -> tuple[pd.DataFrame, pd.DataFrame]:

    #Returns:
        #stats_df: Regular season game data from team perspective.
        #Teams_df: Team metadata (team names, IDs, etc.)

    # === 1.) Validate and Load Data Directory ===
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Data directory not found: {data_dir}")

    Teams_df = pd.read_csv(os.path.join(data_dir, "MTeams.csv"))
    RegularSeasonDetailedResults_df = pd.read_csv(os.path.join(data_dir, "MRegularSeasonDetailedResults.csv"))
    MasseyOrdinals = pd.read_csv(os.path.join(data_dir, "MMasseyOrdinals.csv"))
    NCAATourneySeeds_df = pd.read_csv(os.path.join(data_dir, "MNCAATourneySeeds.csv"))

    # === 2.) Filter Regular Season Data for Specified Start/End Years ===
    regular_df = RegularSeasonDetailedResults_df[
        RegularSeasonDetailedResults_df["Season"].between(start_year, end_year)
    ]

    cols_to_keep = [
        "Season", "DayNum",
        "WTeamID", "WScore", "LTeamID", "LScore",
        "WFGM", "WFGA", "WFGM3", "WFGA3", "WFTM", "WFTA",
        "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF",
        "LFGM", "LFGA", "LFGM3", "LFGA3", "LFTM", "LFTA",
        "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"
    ]
    regular_df = regular_df[cols_to_keep]

    # === 3.) Transform Data into Team Perspective ===
    def build_team_df(df: pd.DataFrame, is_win: bool) -> pd.DataFrame:
        prefix = "W" if is_win else "L"
        opp_prefix = "L" if is_win else "W"
        renamed = {
            f"{prefix}TeamID": "TeamID",
            f"{opp_prefix}TeamID": "OppTeamID",
            f"{prefix}Score": "PointsFor",
            f"{opp_prefix}Score": "PointsAgainst",
            f"{prefix}FGM": "FGM", f"{prefix}FGA": "FGA",
            f"{prefix}FGM3": "FGM3", f"{prefix}FGA3": "FGA3",
            f"{prefix}FTM": "FTM", f"{prefix}FTA": "FTA",
            f"{prefix}OR": "OR", f"{prefix}DR": "DR",
            f"{prefix}Ast": "Ast", f"{prefix}TO": "TO",
            f"{prefix}Stl": "Stl", f"{prefix}Blk": "Blk", f"{prefix}PF": "PF",
            f"{opp_prefix}FGM": "OppFGM", f"{opp_prefix}FGA": "OppFGA",
            f"{opp_prefix}FGM3": "OppFGM3", f"{opp_prefix}FGA3": "OppFGA3",
            f"{opp_prefix}FTM": "OppFTM", f"{opp_prefix}FTA": "OppFTA",
            f"{opp_prefix}OR": "OppOR", f"{opp_prefix}DR": "OppDR",
            f"{opp_prefix}Ast": "OppAst", f"{opp_prefix}TO": "OppTO",
            f"{opp_prefix}Stl": "OppStl", f"{opp_prefix}Blk": "OppBlk",
            f"{opp_prefix}PF": "OppPF"
        }
        cols = ["Season", "DayNum"] + list(renamed.keys())
        df_team = df[cols].copy().rename(columns=renamed)
        df_team["IsWin"] = int(is_win)
        df_team["IsLoss"] = int(not is_win)
        return df_team

    df_win = build_team_df(regular_df, is_win=True)
    df_loss = build_team_df(regular_df, is_win=False)
    stats_df = pd.concat([df_win, df_loss], ignore_index=True)

    # === 4.) Add External Rankings (e.g., KenPom) ===
    rankings = MasseyOrdinals[
        (MasseyOrdinals["SystemName"].isin(systems)) &
        (MasseyOrdinals["RankingDayNum"] == ranking_day)
    ]
    externalRanks = rankings.pivot_table(
        index=["Season", "TeamID"],
        columns="SystemName",
        values="OrdinalRank",
        aggfunc="mean"
    ).reset_index()

    col_rename = {system: "KenPomRank" if system == "POM" else f"{system}Rank" for system in systems}
    externalRanks.rename(columns=col_rename, inplace=True)

    stats_df = stats_df.merge(externalRanks, on=["Season", "TeamID"], how="left")

    # === 5.) Add NCAA Tournament Seeds ===
    seeds_df = NCAATourneySeeds_df.copy()
    seeds_df["SeedNumber"] = seeds_df["Seed"].str[1:3].astype(int)

    stats_df = stats_df.merge(
        seeds_df[["Season", "TeamID", "SeedNumber"]],
        on=["Season", "TeamID"], how="left"
    )

    return stats_df, Teams_df


#### Data Preprocessing Function
- Calculating Key Metrics (i.e., FG%, Assist Ratio, Efficiency, Rolling Avgss, etc.)

In [3]:
def preprocess_team_stats(df: pd.DataFrame, window: int = 5) -> pd.DataFrame:
    # Returns: dataframe w/ advanced statistics like effificncy and rolling averages

    # === 1.) Game-by-Game Advanced Stats ===
    df["eFGPct"] = ((df["FGM"] + 0.5 * df["FGM3"]) / df["FGA"]).fillna(0)
    df["OppeFGPct"] = ((df["OppFGM"] + 0.5 * df["OppFGM3"]) / df["OppFGA"]).fillna(0)

    df["TotalReb"] = df["OR"] + df["DR"]
    df["OppTotalReb"] = df["OppOR"] + df["OppDR"]

    for col in ["Ast", "Stl", "Blk", "TO", "PF", "OppAst", "OppStl", "OppBlk", "OppTO", "OppPF"]:
        df[col] = df[col].fillna(0)

    df["AssistRate"] = (df["Ast"] / df["FGM"].where(df["FGM"] != 0, 1)).fillna(0)
    df["OppAssistRate"] = (df["OppAst"] / df["OppFGM"].where(df["OppFGM"] != 0, 1)).fillna(0)

    df["TeamPoss"] = df["FGA"] - df["OR"] + df["TO"] + (0.44 * df["FTA"])
    df["OppPoss"] = df["OppFGA"] - df["OppOR"] + df["OppTO"] + (0.44 * df["OppFTA"])

    df["OffEff"] = (df["PointsFor"] / df["TeamPoss"]).fillna(0) * 100
    df["DefEff"] = (df["PointsAgainst"] / df["OppPoss"]).fillna(0) * 100

    df["TORate"] = (df["TO"] / df["TeamPoss"]).fillna(0)
    df["OppTORate"] = (df["OppTO"] / df["OppPoss"]).fillna(0)

    df["FTRate"] = (df["FTA"] / df["FGA"]).fillna(0)
    df["OppFTRate"] = (df["OppFTA"] / df["OppFGA"]).fillna(0)

    df["ORate"] = (df["OR"] / (df["OR"] + df["OppDR"]).replace(0, 1)).fillna(0)
    df["OppORate"] = (df["OppOR"] / (df["OppOR"] + df["DR"]).replace(0, 1)).fillna(0)

    # === 2.) Rolling Averages (Getting Avg. for Previous <window> games) ===
    df.sort_values(by=["Season", "TeamID", "DayNum"], inplace=True)

    cols_for_rolling = [
        "PointsFor", "PointsAgainst",
        "TeamPoss", "OppPoss",
        "eFGPct", "OppeFGPct",
        "AssistRate", "OppAssistRate",
        "OffEff", "DefEff",
        "TORate", "OppTORate",
        "FTRate", "OppFTRate",
        "ORate", "OppORate"
    ]

    def rolling_features_exclude_current(group, cols, window):
        group_shifted = group[cols].shift(1)
        group_rolled = group_shifted.rolling(window=window, min_periods=1).mean()
        new_col_names = {c: f"{c}Rol{window}" for c in cols}
        group_rolled.rename(columns=new_col_names, inplace=True)
        return pd.concat([group, group_rolled], axis=1)

    df = (
        df.groupby(["Season", "TeamID"], group_keys=False)
        .apply(lambda g: rolling_features_exclude_current(g, cols_for_rolling, window))
    )

    return df


#### Add Tournament Matchup Function
- Used only in training to bring in Tournament Matchups and Results to measure accuracy

In [4]:
def prepare_tournament_matchups(df_stats: pd.DataFrame, data_path, start_year: int = 2015, end_year: int = 2024,) -> pd.DataFrame:
    
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"Data directory not found: {data_path}")

    df_tourney_raw = pd.read_csv(os.path.join(data_path, "MNCAATourneyCompactResults.csv"))

    df_tourney = df_tourney_raw[df_tourney_raw["Season"].between(start_year, end_year)].copy()
    df_tourney.rename(columns={
        "WTeamID": "TeamA",
        "LTeamID": "TeamB",
        "WScore": "ScoreA",
        "LScore": "ScoreB"
    }, inplace=True)
    df_tourney["A_Win"] = 1

    df_copy = df_tourney.copy()
    df_copy["A_Win"] = 0
    df_copy[["TeamA", "TeamB", "ScoreA", "ScoreB"]] = df_copy[["TeamB", "TeamA", "ScoreB", "ScoreA"]]

    df_symm = pd.concat([df_tourney, df_copy], ignore_index=True)
    df_symm.sort_values("DayNum", inplace=True)

    # Merge Team A rolling stats
    df_symm["dummyTeamA"] = df_symm["TeamA"]
    df_regular = df_stats.copy()
    df_regular.rename(columns={"TeamID": "dummyTeamA"}, inplace=True)

    df_symmA = pd.merge_asof(
        df_symm,
        df_regular.sort_values("DayNum"),
        left_on="DayNum",
        right_on="DayNum",
        by=["Season", "dummyTeamA"],
        direction="backward"
    )

    regular_cols_A = [
        c for c in df_regular.columns if c not in ["Season", "dummyTeamA", "DayNum"]
    ]
    df_symmA.rename(columns={c: f"A_{c}" for c in regular_cols_A}, inplace=True)

    # Merge Team B rolling stats
    df_regular.rename(columns={"dummyTeamA": "TeamID"}, inplace=True)
    df_regular.rename(columns={"TeamID": "dummyTeamB"}, inplace=True)

    df_symmA["dummyTeamB"] = df_symmA["TeamB"]

    df_symmB = pd.merge_asof(
        df_symmA.sort_values("DayNum"),
        df_regular.sort_values("DayNum"),
        left_on="DayNum",
        right_on="DayNum",
        by=["Season", "dummyTeamB"],
        direction="backward"
    )

    regular_cols_B = [
        c for c in df_regular.columns if c not in ["Season", "dummyTeamB", "DayNum"]
    ]
    df_symmB.rename(columns={c: f"B_{c}" for c in regular_cols_B}, inplace=True)

    df_symmB.drop(columns=["dummyTeamA", "dummyTeamB"], errors="ignore", inplace=True)

    return df_symmB


In [5]:
data_path = "/Users/jackthorp/Documents/MarchMadness/Version 01/Datasets/Kaggle March Madness Data"
print('Getting Regular Season Data...')
regular_season_df, teams_df = build_team_season_df(data_dir=data_path, start_year = 2015, end_year = 2024, systems = ['POM'])
print('Successfully captured datasets for analysis. Now Preprocessing data...')
processed_df = preprocess_team_stats(regular_season_df, window=5)
print('Succesfully Preprocessed data. Now adding NCAA Tournament data to judge predictions...')
df_final = prepare_tournament_matchups(processed_df, data_path, start_year = 2015, end_year = 2024)
print('Successfully added NCAA Tournament data')

# Label IVs and DVs
feature_cols = [
    # Team A Stats:
    "A_PointsForRol5", "A_PointsAgainstRol5",
    "A_TeamPossRol5", "A_OppPossRol5",
    "A_eFGPctRol5", "A_OppeFGPctRol5",
    "A_AssistRateRol5", "A_OppAssistRateRol5",
    "A_OffEffRol5", "A_DefEffRol5",
    "A_TORateRol5", "A_OppTORateRol5",
    "A_FTRateRol5", "A_OppFTRateRol5",
    "A_ORateRol5", "A_OppORateRol5",
    "A_KenPomRank",
    "A_SeedNumber",

    # Team B Stats:
    "B_PointsForRol5", "B_PointsAgainstRol5",
    "B_TeamPossRol5", "B_OppPossRol5",
    "B_eFGPctRol5", "B_OppeFGPctRol5",
    "B_AssistRateRol5", "B_OppAssistRateRol5",
    "B_OffEffRol5", "B_DefEffRol5",
    "B_TORateRol5", "B_OppTORateRol5",
    "B_FTRateRol5", "B_OppFTRateRol5",
    "B_ORateRol5", "B_OppORateRol5",
    "B_KenPomRank",
    "B_SeedNumber"
]

target_col = "A_Win"

X = df_final[feature_cols].fillna(0)
y = df_final[target_col]

print('Splitting Dataframe into Train & Test Datasets')
#Train Test Split
df_train = df_final[df_final["Season"].between(2015, 2022)]
df_test  = df_final[df_final["Season"].between(2023, 2024)]

X_train = df_train[feature_cols].fillna(0)
y_train = df_train["A_Win"]

X_test = df_test[feature_cols].fillna(0)
y_test = df_test["A_Win"]

print('Training Random Forest Regression Model!')
# Train Random Forest Model
model = RandomForestClassifier(
    n_estimators=10000, 
    max_features="sqrt",
    random_state=42
)
model.fit(X_train, y_train)

# Evaluate Model
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Validation Accuracy: {acc:.3f}")
print(classification_report(y_test, y_pred))

Getting Regular Season Data...
Successfully captured datasets for analysis. Now Preprocessing data...


/var/folders/9y/cfbm31fn0xq1085v9zqymwf00000gp/T/ipykernel_83120/2513021372.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: rolling_features_exclude_current(g, cols_for_rolling, window))


Succesfully Preprocessed data. Now adding NCAA Tournament data to judge predictions...
Successfully added NCAA Tournament data
Splitting Dataframe into Train & Test Datasets
Training Random Forest Regression Model!
Validation Accuracy: 0.724
              precision    recall  f1-score   support

           0       0.73      0.72      0.72       134
           1       0.72      0.73      0.73       134

    accuracy                           0.72       268
   macro avg       0.72      0.72      0.72       268
weighted avg       0.72      0.72      0.72       268



#### defining testing function

In [9]:
def simulate_seeded_matchups(processed_df: pd.DataFrame, teams_df: pd.DataFrame, model, season: int = 2025) -> pd.DataFrame:
    # 1. Get all teams with a seed (i.e., predicted tournament teams)
    seeded_teams = processed_df[(processed_df["Season"] == season) & (~processed_df["SeedNumber"].isna())]
    
    # only grab most recent team result (important for rolling statistics)
    seeded_teams = (
        seeded_teams.sort_values("DayNum")
        .groupby("TeamID", as_index=False)
        .last()
    )

    team_ids = seeded_teams["TeamID"].unique()

    # 2. Generate unique matchups (A vs B, not both A vs B AND B vs A)
    matchups = list(combinations(team_ids, 2))
    df_matchups = pd.DataFrame(matchups, columns=["TeamA", "TeamB"])
    df_matchups["Season"] = season
    df_matchups["DayNum"] = 133

    # 3. Prepare rolling stats
    df_regular = processed_df.copy()
    df_regular["dummyTeamID"] = df_regular["TeamID"]

    # Keep latest stat for each team
    df_regular_latest = (
        df_regular[df_regular["Season"] == season]
        .sort_values("DayNum")
        .groupby("TeamID", as_index=False)
        .last()
    )

    # 4. Merge Team A stats
    df_matchups["dummyTeamA"] = df_matchups["TeamA"]
    dfA = pd.merge(
        df_matchups,
        df_regular_latest.rename(columns={"TeamID": "dummyTeamA"}),
        on=["dummyTeamA"],
        how="left"
    )
    dfA.rename(columns={c: f"A_{c}" for c in df_regular_latest.columns if c not in ["TeamID", "DayNum"]}, inplace=True)

    # 5. Merge Team B stats
    dfA["dummyTeamB"] = dfA["TeamB"]
    df_final = pd.merge(
        dfA,
        df_regular_latest.rename(columns={"TeamID": "dummyTeamB"}),
        on=["dummyTeamB"],
        how="left"
    )
    df_final.rename(columns={c: f"B_{c}" for c in df_regular_latest.columns if c not in ["TeamID", "DayNum"]}, inplace=True)

    # 6. Predict
    X = df_final[feature_cols].fillna(0)
    df_final["Predicted_A_Win_Prob"] = model.predict_proba(X)[:, 1].round(2)
    df_final["Predicted_A_Win"] = model.predict(X)

    # 7. Merge team names
    team_names = teams_df.copy()
    df_final = df_final.merge(team_names[["TeamID", "TeamName"]], left_on="TeamA", right_on="TeamID", how="left").rename(columns={"TeamName": "TeamA_Name"}).drop(columns="TeamID")
    df_final = df_final.merge(team_names[["TeamID", "TeamName"]], left_on="TeamB", right_on="TeamID", how="left").rename(columns={"TeamName": "TeamB_Name"}).drop(columns="TeamID")

    return df_final


In [10]:
data_path = "/Users/jackthorp/Documents/MarchMadness/Version 01/Datasets/Kaggle March Madness Data"
print('Getting Regular Season Data...')
regular_season_df_2025, teams_df = build_team_season_df(data_dir=data_path, start_year = 2025, end_year = 2025, systems = ['POM'])
print('Successfully captured datasets for analysis. Now Preprocessing data...')
processed_df_2025 = preprocess_team_stats(regular_season_df_2025, window=5)
print('Succesfully Preprocessed data. Time to run model against recent data!')

# Only run model for every combination of teams that are seeded in 2025 march madness tournament
df_final_2025 = simulate_seeded_matchups(processed_df_2025, teams_df, model, season=2025)

df_export = df_final_2025[['TeamA_Name', 'TeamB_Name', 'Predicted_A_Win_Prob', 'Predicted_A_Win']]
excel_file_path = os.path.expanduser("~/Downloads/2025_Predictions.xlsx")
df_export.to_excel(excel_file_path, index=False)
print('successfully downloaded model results')

Getting Regular Season Data...
Successfully captured datasets for analysis. Now Preprocessing data...


/var/folders/9y/cfbm31fn0xq1085v9zqymwf00000gp/T/ipykernel_83120/2513021372.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: rolling_features_exclude_current(g, cols_for_rolling, window))


Succesfully Preprocessed data. Time to run model against recent data!
